# Исследование для телекоммуникационной компании.

# Цель исследования- задача определить текущий уровень потребительской лояльности, или NPS среди клиентов из России.

<h1>План исследования.<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Выгрузить-данные-из-БД-и-создать-датафрейм-соответствующий-ТЗ" data-toc-modified-id="Выгрузить-данные-из-БД-и-создать-датафрейм-соответствующий-ТЗ-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Выгрузить данные из БД и создать датафрейм соответствующий ТЗ</a></span></li><li><span><a href="#Исследовать-и-подготовить-данные-для-создания-.csv-таблицы" data-toc-modified-id="Исследовать-и-подготовить-данные-для-создания-.csv-таблицы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследовать и подготовить данные для создания .csv таблицы</a></span></li><li><span><a href="#Ссылка-на-дашборд-на-сайте-Tableau-Public:" data-toc-modified-id="Ссылка-на-дашборд-на-сайте-Tableau-Public:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ссылка на дашборд на сайте Tableau Public:</a></span></li><li><span><a href="#Ссылка-на-pdf-файл-с-презентацией:" data-toc-modified-id="Ссылка-на-pdf-файл-с-презентацией:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Ссылка на pdf-файл с презентацией:</a></span></li></ul></div>

### Выгрузить данные из БД и создать датафрейм соответствующий ТЗ

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
WITH u AS
  (SELECT user_id,
          lt_day,
          (CASE
               WHEN lt_day <= 365 THEN 'new'
               WHEN lt_day > 365 THEN 'not_new'
           END) AS is_new,
          age,
          (CASE
               WHEN gender_segment=1 THEN 'female'
               WHEN gender_segment=0 THEN 'male'
           END) AS gender_segment,
          os_name,
          cpe_type_name,
          nps_score,
          (CASE
               WHEN nps_score>=9 THEN 'promoters'
               WHEN nps_score>=7
                    AND nps_score<9 THEN 'passives'
               ELSE 'detractors'
           END) AS nps_group,
          location_id,
          age_gr_id,
          tr_gr_id,
          lt_gr_id
   FROM USER),
     l AS
  (SELECT country,
          city,
          location_id
   FROM LOCATION),
     a_s AS
  (SELECT title,
          age_gr_id
   FROM age_segment),
     t_s AS
  (SELECT title,
          tr_gr_id
   FROM traffic_segment),
     l_s AS
  (SELECT title,
          lt_gr_id
   FROM lifetime_segment)
SELECT u.user_id,
       u.lt_day,
       u.is_new,
       u.age,
       u.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       substr(a_s.title,4) AS age_segment,
       substr(t_s.title, 4) AS traffic_segment,
       substr(l_s.title,4) AS lifetime_segment,
       u.nps_score,
       u.nps_group
FROM u
JOIN l ON u.location_id=l.location_id
JOIN a_s ON u.age_gr_id=a_s.age_gr_id
JOIN t_s ON u.tr_gr_id=t_s.tr_gr_id
JOIN l_s ON u.lt_gr_id=l_s.lt_gr_id

"""

In [4]:
df = pd.read_sql(query, engine)
df.sample(10)


,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
482117,YXZGRY,3198,not_new,54.0,male,ANDROID,SMARTPHONE,Россия,СанктПетербург,45-54,10-15,36+,8,passives
241272,MH66VL,2874,not_new,41.0,female,ANDROID,SMARTPHONE,Россия,Екатеринбург,35-44,5-10,36+,10,promoters
129892,GP45JW,5179,not_new,54.0,female,ANDROID,SMARTPHONE,Россия,Казань,45-54,1-5,36+,10,promoters
321390,QMTW2C,4172,not_new,59.0,female,ANDROID,SMARTPHONE,Россия,Казань,55-64,5-10,36+,10,promoters
463182,XYPOP2,176,new,46.0,female,ANDROID,TABLET,Россия,НижнийНовгород,45-54,1-5,4-6,10,promoters
9856,AI9A1Z,308,new,51.0,male,ANDROID,SMARTPHONE,Россия,Москва,45-54,10-15,7-12,10,promoters
67247,DGREVJ,1471,not_new,43.0,male,ANDROID,SMARTPHONE,Россия,Новосибирск,35-44,30-35,36+,10,promoters
363498,SSVBFU,5501,not_new,41.0,female,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,detractors
316918,QEDP8J,540,not_new,28.0,female,ANDROID,SMARTPHONE,Россия,НижнийНовгород,25-34,15-20,13-24,10,promoters
269608,NY0X8B,48,new,39.0,female,IOS,SMARTPHONE,Россия,Москва,35-44,70-75,2,10,promoters


### Исследовать и подготовить данные для создания .csv таблицы

In [5]:
df.to_csv('telecomm_csi_tableau_fix.csv', index=False)

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/hookjr/viz/2_16656429995420/Dashboard1?publish=yes


### Ссылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/lbUbmdIu7Mva5w
